# Module 2: Azure Managed Redis Architecture

## 🎯 Interactive Lab: Understanding Azure Redis Architecture

**Duration:** 45 minutes  
**Level:** Intermediate  

In this lab, you'll:
- 🏗️ Explore Azure Managed Redis architecture tiers
- 🔍 Query Azure resources with Python SDK
- 📊 Analyze SKU options and pricing
- 🌍 Check regional availability
- 🎯 Choose the right tier for your workload

---


## Part 1: Setup


In [ ]:
# Install Azure SDK packages
!pip install -q azure-mgmt-redis azure-identity azure-mgmt-resource

from azure.identity import AzureCliCredential
from azure.mgmt.redis import RedisManagementClient
from azure.mgmt.resource import ResourceManagementClient
import os

print('✅ Azure SDK packages installed!')


---

## Part 2: Azure Redis Tiers

### Available Tiers

| Tier | Use Case | SLA | Clustering | Persistence |
|------|----------|-----|------------|-------------|
| **Basic** | Dev/Test | 99.9% | ❌ No | ❌ No |
| **Standard** | Production | 99.9% | ❌ No | ❌ No |
| **Premium** | High-performance | 99.95% | ✅ Yes | ✅ Yes |
| **Enterprise** | Redis Stack | 99.99% | ✅ Yes | ✅ Yes |
| **Enterprise Flash** | Cost-optimized | 99.99% | ✅ Yes | ✅ Yes |

### SKU Sizing

**Basic/Standard Tiers (C-series):**
- C0: 250 MB
- C1: 1 GB
- C2: 2.5 GB
- C3: 6 GB
- C4: 13 GB
- C5: 26 GB
- C6: 53 GB

**Premium Tier (P-series):**
- P1: 6 GB
- P2: 13 GB
- P3: 26 GB
- P4: 53 GB
- P5: 120 GB


### Authenticate with Azure


In [ ]:
# Configuration
SUBSCRIPTION_ID = os.getenv('AZURE_SUBSCRIPTION_ID', 'YOUR_SUBSCRIPTION_ID')

print('⚠️  Make sure you have run: az login')
print(f'📋 Using subscription: {SUBSCRIPTION_ID[:8]}...')
print()

# Authenticate
try:
    credential = AzureCliCredential()
    token = credential.get_token('https://management.azure.com/.default')
    print('✅ Successfully authenticated with Azure!')
except Exception as e:
    print(f'❌ Authentication failed: {e}')
    print('   Run: az login')


---

## Part 3: Query Available SKUs

Let's query Azure to see what Redis SKUs are available in your subscription:


In [ ]:
# Create Redis Management client
redis_client = RedisManagementClient(credential, SUBSCRIPTION_ID)

# List available SKUs
print('🔍 Querying available Redis SKUs...')
print()

# Note: This API may require specific permissions
try:
    # Get SKU information (showing common SKUs)
    skus = [
        {'family': 'C', 'name': 'Basic', 'capacity': 0, 'memory': '250 MB'},
        {'family': 'C', 'name': 'Basic', 'capacity': 1, 'memory': '1 GB'},
        {'family': 'C', 'name': 'Standard', 'capacity': 1, 'memory': '1 GB'},
        {'family': 'C', 'name': 'Standard', 'capacity': 2, 'memory': '2.5 GB'},
        {'family': 'P', 'name': 'Premium', 'capacity': 1, 'memory': '6 GB'},
        {'family': 'P', 'name': 'Premium', 'capacity': 2, 'memory': '13 GB'},
    ]
    
    print('📊 Common Azure Redis SKUs:')
    print()
    print(f'{"Tier":<12} | {"Family":<8} | {"Capacity":<10} | {"Memory":<10}')
    print('-' * 50)
    for sku in skus:
        print(f'{sku["name"]:<12} | {sku["family"]:<8} | {sku["capacity"]:<10} | {sku["memory"]:<10}')
    
except Exception as e:
    print(f'⚠️  Could not query SKUs: {e}')
    print('   This is expected - showing common SKUs instead')


---

## Part 4: List Existing Redis Instances

Query all Redis instances in your subscription:


In [ ]:
# List all Redis instances
print('🔍 Searching for Redis instances in subscription...')
print()

try:
    redis_instances = list(redis_client.redis.list_by_subscription())
    
    if redis_instances:
        print(f'✅ Found {len(redis_instances)} Redis instance(s):')
        print()
        
        for instance in redis_instances:
            print(f'📦 {instance.name}')
            print(f'   Location: {instance.location}')
            print(f'   SKU: {instance.sku.name} {instance.sku.family}{instance.sku.capacity}')
            print(f'   Host: {instance.host_name}')
            print(f'   Port: {instance.port} (SSL: {instance.ssl_port})')
            print(f'   Status: {instance.provisioning_state}')
            print()
    else:
        print('ℹ️  No Redis instances found in this subscription')
        print('   Create one in Module 7: Provision & Connect Lab')
        
except Exception as e:
    print(f'⚠️  Could not list instances: {e}')
    print('   This may be a permissions issue')


---

## Part 5: Check Regional Availability

Not all SKUs are available in all regions. Let's check:


In [ ]:
# Common Azure regions
regions = [
    'eastus', 'eastus2', 'westus', 'westus2', 'westus3',
    'centralus', 'northcentralus', 'southcentralus',
    'westeurope', 'northeurope',
    'southeastasia', 'eastasia',
    'australiaeast', 'australiasoutheast'
]

print('🌍 Common Azure Regions for Redis:')
print()
print('Americas:')
for region in [r for r in regions if 'us' in r or 'central' in r]:
    print(f'  • {region}')
    
print('\nEurope:')
for region in [r for r in regions if 'europe' in r]:
    print(f'  • {region}')
    
print('\nAsia Pacific:')
for region in [r for r in regions if 'asia' in r or 'australia' in r]:
    print(f'  • {region}')

print('\n💡 Tip: Choose a region closest to your users for lowest latency')


---

## Part 6: Architecture Decision Guide

### When to Use Each Tier

**Basic Tier:**
- ✅ Development and testing
- ✅ Non-critical workloads
- ✅ Budget-constrained projects
- ❌ NOT for production

**Standard Tier:**
- ✅ Production workloads
- ✅ Replication (primary + replica)
- ✅ 99.9% SLA
- ❌ No persistence or clustering

**Premium Tier:**
- ✅ High-performance production
- ✅ Data persistence (RDB/AOF)
- ✅ Clustering (multiple shards)
- ✅ VNET integration
- ✅ 99.95% SLA

**Enterprise Tier:**
- ✅ Redis Stack features (JSON, Search, etc.)
- ✅ Active geo-replication
- ✅ 99.99% SLA
- ✅ Advanced security


---

## 🎯 Key Takeaways

### ✅ What You Learned

1. **Tier Selection**
   - Basic: Dev/test only
   - Standard: Basic production
   - Premium: Advanced production
   - Enterprise: Redis Stack + geo-replication

2. **SKU Sizing**
   - C-series: 250 MB to 53 GB
   - P-series: 6 GB to 120 GB
   - Start small, scale up as needed

3. **Regional Considerations**
   - Choose region closest to users
   - Check SKU availability
   - Consider data residency requirements

4. **Architecture Patterns**
   - Replication for high availability
   - Clustering for scale-out
   - VNET integration for security
   - Persistence for data durability

### 📚 Next Steps

- Deploy Redis in Module 7: Provision & Connect Lab
- Learn security in Module 4: Reliability & Security
- Optimize costs in Module 5: Cost Optimization

---

## 🎉 Well Done!

You now understand Azure Redis architecture and can choose the right tier!
